In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

In [3]:
from data_prep_functions import mnist_prep
from plotting_functions import nn_plot_epoch_acc
from enkf_functions import enkf_classifier
from saving_functions import load_objects
from model_functions import nn_load
import reproducible
import no_gpu

In [4]:
X_train, X_test, y_train, y_test = mnist_prep()

In [5]:
layers = 5
neurons = [128, 128, 64, 32, 10]

In [6]:
setting_dict = {"particles": 1,
                "epochs": 10,
                "batch_size": 1000,    # len(X_train)
                "h_0": 2,
                "delta": 0.005,
                "epsilon": 0.5,
                "randomization": False,
                "shuffle": True,
                "early_stopping": False,
                "early_stopping_diff": 0.001
               }

In [7]:
particle_list = [750]

In [8]:
import time

In [ ]:
for p in particle_list:
    setting_dict["particles"] = p
    
    import random
    import os
    import numpy as np
    import tensorflow as tf

    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(42)
    random.seed(42)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
    tf.compat.v1.set_random_seed(42)
    sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    
    start_time = time.time()
    mean_model, mean_model_train_acc, mean_model_test_acc = enkf_classifier(X_train,
                                                                            X_test,
                                                                            y_train,
                                                                            y_test,
                                                                            layers,
                                                                            neurons,
                                                                            setting_dict,
                                                                            save_all = True,
                                                                            file_var = "../objects/mnist/enkf_mnist_full_E{}_B{}_P{}_H{}.pckl".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]),
                                                                            file_model = "../models/mnist/enkf_mnist_full_E{}_B{}_P{}_H{}.h5".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]),
                                                                            verbose = 1)
    end_time = time.time()
    print("Calculation time for {} particles: {} minutes.".format(p, (end_time - start_time) / 60))
    
    nn_plot_epoch_acc(mean_model_train_acc,
                      mean_model_test_acc,
                      mean_comparison = None,
                      savefig = True,
                      file = "../img/mnist/enkf_mnist_full_E{}_B{}_P{}_H{}.png".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))

Epoch 1. Training Accuracy: 0.435, Test Accuracy: 0.456.
Epoch 2. Training Accuracy: 0.669, Test Accuracy: 0.678.
Epoch 3. Training Accuracy: 0.689, Test Accuracy: 0.697.
Epoch 4. Training Accuracy: 0.705, Test Accuracy: 0.715.
Epoch 5. Training Accuracy: 0.722, Test Accuracy: 0.732.


In [ ]:
obj_dict = load_objects("../objects/mnist/enkf_mnist_full_E{}_B{}_P{}_H{}.pckl".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))

In [ ]:
final_model = nn_load("../models/mnist/enkf_mnist_full_E{}_B{}_P{}_H{}.h5".format(setting_dict["epochs"], setting_dict["batch_size"], setting_dict["particles"], setting_dict["h_0"]))